In [2]:
# 🔄 Auto-reload modules when their source code changes
%load_ext autoreload
%autoreload 2

# 📦 Set up src/ and utils/ paths (project root is detected via pyproject.toml)
%run ../bootstrap.py

In [3]:
import requests
from datetime import datetime, timedelta, timezone
import json
from pathlib import Path
from dotenv import load_dotenv
import os
import time
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, clear_output

from sleeping_beauty.config.config import Config

In [4]:
load_dotenv()

True

In [5]:
config = Config()


PROJECT_ROOT = Path(config.PROJECT_ROOT)
config.config_path = str(PROJECT_ROOT / "configs" / "config.yaml")
# "configs/FE/features_config.yaml"
config.load_from_yaml(config.config_path)




[Config] Loaded YAML config: /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml
[Config] Overriding 'divider': False → True
[Config] Setting 'divider': False → True


In [6]:
config.print_config_info()

📂 Configuration
--------------------------------------------------
Configuration file:       /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml
--------------------------------------------------
🔐 Auth / Oura
--------------------------------------------------
Token path:               /Users/kenneth/.sleeping_beauty/oura_token.json
Client ID set:            True
Client Secret set:        True
Redirect URI:             http://localhost:8400/callback
Scopes:                   ['daily', 'heartrate', 'personal', 'session']


In [7]:
from datetime import date, timedelta
import asyncio

from sleeping_beauty.clients.oura_api_client import OuraApiClient
from sleeping_beauty.oura.auth.oura_auth import OuraAuth  
from sleeping_beauty.oura.auth.storage.file_storage import FileTokenStorage

In [8]:
from sleeping_beauty.oura.auth.domain.auth_preflight_result import AuthPreflightReport


storage = FileTokenStorage(
    path=Path(
        config.oura_token_path
    ).expanduser()
)

# Instantiate your existing auth service
oura_auth = OuraAuth.from_config()
preflight: AuthPreflightReport = oura_auth.preflight_check()
print("\n".join(preflight.messages))

# Token provider callable (what the client expects)
token_provider = oura_auth.get_access_token   ## notice this is passing the get_access_token as the callable function 

# Create the API client
client = OuraApiClient(token_provider=token_provider)

🔐 Oura Auth Preflight Check
----------------------------------------
✅ client_id present
✅ client_secret present
✅ redirect_uri: http://localhost:8400/callback
✅ requested scopes: ['extapi:daily', 'extapi:heartrate', 'extapi:personal', 'extapi:session']
✅ token found in storage
✅ token valid for ~35952 minutes
✅ token covers requested scopes
----------------------------------------
✔️  Preflight check complete


In [ ]:
# ---------------------------------------------------------------------
# Define a small, safe test window
# ---------------------------------------------------------------------

end_date = date.today()
start_date = end_date - timedelta(days=7)

print(f"Testing daily readiness from {start_date} to {end_date}")

# ---------------------------------------------------------------------
# 1. Fetch a single page
# ---------------------------------------------------------------------

page = await client.get_daily_readiness_score_page(
    start_date=start_date,
    end_date=end_date,
)

print("\n=== Page summary ===")
print("Items:", len(page.data))
print("Next token:", page.next_token)

if page.data:
    first = page.data[0]
    print("\n=== First DailyReadinessScore ===")
    print(first)

# ---------------------------------------------------------------------
# 2. Iterate all records
# ---------------------------------------------------------------------

print("\n=== Iterating all daily readiness scores ===")

items = []
async for item in client.iter_daily_readiness_scores(
    start_date=start_date,
    end_date=end_date,
):
    items.append(item)
    print(item.day, "score =", item.score)

print(f"\nTotal records fetched: {len(items)}")

# ---------------------------------------------------------------------
# 3. Fetch single document by ID (if available)
# ---------------------------------------------------------------------

if items:
    doc_id = items[1].id
    print(f"\n=== Fetching single document: {doc_id} ===")

    single = await client.get_daily_readiness_score(document_id=doc_id)

    print(single)

Testing daily readiness from 2026-01-16 to 2026-01-17

=== Page summary ===
Items: 2
Next token: None

=== First DailyReadinessScore ===
DailyReadinessScore(id='cc8218cb-4c58-49ab-9268-9c33b54cf83e', day=datetime.date(2026, 1, 16), score=80, temperature_deviation=0.12, temperature_trend_deviation=-0.12, timestamp='2026-01-16T00:00:00.000+00:00', contributors=ReadinessContributors(activity_balance=71, body_temperature=95, hrv_balance=87, previous_day_activity=66, previous_night=78, recovery_index=88, resting_heart_rate=100, sleep_balance=56, sleep_regularity=90), raw={'id': 'cc8218cb-4c58-49ab-9268-9c33b54cf83e', 'contributors': {'activity_balance': 71, 'body_temperature': 95, 'hrv_balance': 87, 'previous_day_activity': 66, 'previous_night': 78, 'recovery_index': 88, 'resting_heart_rate': 100, 'sleep_balance': 56, 'sleep_regularity': 90}, 'day': '2026-01-16', 'score': 80, 'temperature_deviation': 0.12, 'temperature_trend_deviation': -0.12, 'timestamp': '2026-01-16T00:00:00.000+00:00'})


In [12]:
# ---------------------------------------------------------------------
# Define a small, safe test window
# ---------------------------------------------------------------------

end_date = date(2026,1,11)
start_date = end_date - timedelta(days=1)

print(f"Testing daily readiness from {start_date} to {end_date}")

# ---------------------------------------------------------------------
# 1. Fetch a single page
# ---------------------------------------------------------------------

page = await client.get_daily_readiness_score_page(
    start_date=start_date,
    end_date=end_date,
)

print("\n=== Page summary ===")
print("Items:", len(page.data))
print("Next token:", page.next_token)

if page.data:
    first = page.data[0]
    print("\n=== First DailyReadinessScore ===")
    print(first)

# ---------------------------------------------------------------------
# 2. Iterate all records
# ---------------------------------------------------------------------

print("\n=== Iterating all daily readiness scores ===")

items = []
async for item in client.iter_daily_readiness_scores(
    start_date=start_date,
    end_date=end_date,
):
    items.append(item)
    print(item.day, "score =", item.score)

print(f"\nTotal records fetched: {len(items)}")

# ---------------------------------------------------------------------
# 3. Fetch single document by ID (if available)
# ---------------------------------------------------------------------

if items:
    doc_id = items[1].id
    print(f"\n=== Fetching single document: {doc_id} ===")

    single = await client.get_daily_readiness_score(document_id=doc_id)

    print(single)

Testing daily readiness from 2026-01-10 to 2026-01-11

=== Page summary ===
Items: 2
Next token: None

=== First DailyReadinessScore ===
DailyReadinessScore(id='2777a27a-a98d-4a94-986a-b7ba37b8549a', day=datetime.date(2026, 1, 10), score=69, temperature_deviation=0.13, temperature_trend_deviation=0.07, timestamp='2026-01-10T00:00:00.000+00:00', contributors=ReadinessContributors(activity_balance=78, body_temperature=94, hrv_balance=55, previous_day_activity=65, previous_night=94, recovery_index=76, resting_heart_rate=48, sleep_balance=64, sleep_regularity=79), raw={'id': '2777a27a-a98d-4a94-986a-b7ba37b8549a', 'contributors': {'activity_balance': 78, 'body_temperature': 94, 'hrv_balance': 55, 'previous_day_activity': 65, 'previous_night': 94, 'recovery_index': 76, 'resting_heart_rate': 48, 'sleep_balance': 64, 'sleep_regularity': 79}, 'day': '2026-01-10', 'score': 69, 'temperature_deviation': 0.13, 'temperature_trend_deviation': 0.07, 'timestamp': '2026-01-10T00:00:00.000+00:00'})

===